# Accelerating Zero-Knowledge Proof Systems with Specialized Hardware: The Role of FPGA and ASIC in Enhancing Goldreich-Krawczyk-Rackoff (GKR) Protocols

## Introduction

In the evolving landscape of cryptographic protocols, zero-knowledge proof (ZKP) systems stand out due to their ability to verify the correctness of information without revealing the information itself. A particular type of ZKP, the Goldreich-Krawczyk-Rackoff (GKR) protocol, has shown promising applications in privacy-preserving computations and secure multi-party computations. However, as the complexity and scale of these applications grow, so does the demand for more efficient computation methods. This is where specialized hardware like Field Programmable Gate Arrays (FPGA) and Application-Specific Integrated Circuits (ASIC) come into play.

## The Need for Speed in ZKP

ZKPs are computationally intensive and require significant resources to execute, especially in scenarios involving complex data sets and high-frequency verification demands. The GKR protocol, while efficient in theoretical terms, can benefit greatly from hardware acceleration to meet practical performance requirements.

## FPGA and ASIC: Catalysts for Enhanced GKR Performance

FPGAs and ASICs provide customizable, high-performance computing solutions that can be specifically tailored to enhance the GKR protocol. Here’s how these technologies can make a difference:

1. **Customization for Optimization**: FPGAs allow for the programming of hardware to execute specific tasks such as the polynomial evaluations and interactive proofs required in GKR. ASICs take this a step further by offering optimized, fixed-functionality chips designed for executing GKR operations at even higher speeds and lower power consumption.

2. **Parallel Processing Capabilities**: Both FPGA and ASIC technologies excel in handling parallel processes. This is crucial for scaling ZKP computations, enabling simultaneous processing of multiple proof validations or generation tasks, significantly cutting down the overall computation time.

3. **Energy Efficiency**: With the increasing emphasis on sustainable computing, the energy efficiency of FPGAs and ASICs is a significant advantage. These technologies consume less power compared to general-purpose processors, especially when tuned for specific tasks like those involved in GKR.


## Limitation? Accelerate GKR in Linear Time

The problem of original Sum-check protocol used in [Tha'13] leads to $O(2^{2\cdot log(n)}) = O(n^2)$ time complexity, which is not applicable for hardware implementation. A notable advancement in the optimization of GKR protocols comes from exploiting the inherent sparsity found in arbitrary circuits. According to research presented in the [XZZ'19], by strategically analyzing and utilizing the sparse characteristics of circuits, it is possible to significantly enhance the efficiency of GKR computations. This approach focuses an technique that break Sum-check computation (consists of multiple round of Multi-linear Extension, MLE evaluation) into two phases, which leads to construct two small bookkeeping tables in $O(n)$ time. As a result, the original $O(n^2)$ problem breaks into two $O(n)$ problems, making hardware specialization practical.

The key to this method lies in a careful construction of bookkeeping table before each Sum-check phase by exploiting the sparsity between each neighboring layers throughout GKR protocol. By employing linear-time algorithms that adapt to the sparsity levels of the circuits, the prover phase of the GKR can be optimized. This results in a more streamlined process that not only speeds up the computation but also minimizes the resources required for generating and verifying proofs.

This technique is particularly effective for large-scale applications where the arbitrary circuit complexity can otherwise lead to bottlenecks. By harnessing the sparsity, systems can achieve linear scalability in terms of both time and space, making it feasible to implement more complex and secure zero-knowledge proofs in practical, real-world systems.


### What is the upper bound?

Due to the nature of spatial locality of GKR protocol, an example hardware solution can be like below. 

![Figure 1](hw-example.png) 

GKR Accelerator contains a flatten circuit structure with two small physical SRAMs, as the physical storage of two bookkeeping tables. The neighboring two GKR layers can be streamed into GKR accelerator. For each round of Sum-check protocol, GKR accelerator requests one random challenge via hard-wired logics (or just tiny CPU) from *Proof Transcript SRAM* and proceed to next round.

We can do a simple calculation to study the upper bound on an example arbitrary circuit with 100 layers, 1024 gates per layer. Assuming the MLE of each round Sum-check are completely flattened and unlimited fast, the bottleneck becomes the number of time that GKR accelerator request for random oracle from Proof transcript SRAM. The number of random oracle request time is:

$$
100\ layers \times (2 \times log_{2}(1024) + 2) = 2,200\ challenges
$$

However, this approach is not practical because obviously the bottleneck will become the original hardware implementation for a non-linear Sum-check protocol, where $log_21024 + log_21024 = 20$ variables' multi-linear extension (MLE) to be evaluated, leading to a $2^{20}$ rows bookkeeping table at the first round of Sum-check. However, this problem can be solved by applying the technique from [XZZ'19] where the problem of $O(n^2)$ is broken into two $O(n)$ problems and each problem can be solved by sacrificing a bookkeeping table with $O(n)$ size, leading hardware specialization to be practical.

### Bottleneck Analysis

To conduct a quantitative analysis of the performance bottlenecks in the GKR proof system hardware accelerator, we can create a simple Python simulation. This simulation will estimate the potential bottlenecks in SRAM access times and interface data transfer rates, providing a clearer picture of how these factors might affect the overall system performance.

We'll first define some basic parameters and functions to simulate the hardware behavior, including SRAM access times and data transfer rates. Then, we'll calculate the total time required for the GKR accelerator to process the entire proof, considering the optimizations mentioned.

In [20]:
import numpy as np

# Constants
layers = 100
gates_per_layer = 1024
challenges_per_layer = 2 * np.log2(gates_per_layer) + 2
total_challenges = layers * challenges_per_layer

# Hardware Simulation Parameters
sram_access_time = 0.008  # Time to access SRAM in microseconds
data_transfer_rate = 100  # Data transfer rate in Mbps
challenge_size = 32  # Size of each challenge in bits

def calculate_sram_access_time(total_challenges, access_time):
    """Calculate total time spent on SRAM accesses."""
    return total_challenges * access_time

def calculate_data_transfer_time(total_challenges, challenge_size, transfer_rate):
    """Calculate total time for data transfers."""
    bytes_per_challenge = challenge_size / 8
    total_data = total_challenges * bytes_per_challenge  # Total data in bytes
    return (total_data * 8) / (transfer_rate * 1e6) * 1e6  # Convert time to microseconds

# Calculate times
total_sram_time = calculate_sram_access_time(total_challenges, sram_access_time)
total_transfer_time = calculate_data_transfer_time(total_challenges, challenge_size, data_transfer_rate)

# Print results
print(f"Total SRAM Access Time: {total_sram_time:.2f} microseconds")
print(f"Total Data Transfer Time: {total_transfer_time:.2f} microseconds")


Total SRAM Access Time: 17.60 microseconds
Total Data Transfer Time: 704.00 microseconds


While accessing SRAM and retrieving data indeed consume time, these overheads can be effectively masked within the primary computational phases of the Sum-check algorithm through the use of a ping-pong buffer strategy. This approach allows data to be pre-fetched and processed alternately, enabling continuous computation without idle time. As a result, the time spent on SRAM accesses and data transfers can be considered negligible in the overall performance evaluation. Therefore, we can afford to disregard these overheads and focus our analysis on the multiplication operations, which constitute the primary computational expense in the system.

In [21]:
def calculate_mult_per_layer(layers, gates_per_layer):
    log_gates = int(np.log2(gates_per_layer))
    
    # Step 1: Compute 'hg' table
    # Splitting log_gates into two parts and calculating separately
    first_half = log_gates // 2
    second_half = log_gates - first_half
    
    # Number of multiplications for binary combinations in each half
    mult_first_half = 2 ** first_half * (first_half - 1)
    mult_second_half = 2 ** second_half * (second_half - 1)
    
    # Multiplications to combine results from both halves
    mult_combine_halves = 2 ** first_half * 2 ** second_half
    
    # Total multiplications for step 1
    total_mult_hg = mult_first_half + mult_second_half + mult_combine_halves
    
    # Step 2: Sumcheck for 'x' variables
    # Using the sum of a geometric series to find total multiplications
    total_mult_x = sum(2 ** i for i in range(log_gates - 1, -1, -1))
    
    # Step 3: Sumcheck for 'y' variables, same as 'x'
    total_mult_y = total_mult_x
    
    # Total multiplications for one layer
    total_mult_per_layer = total_mult_hg + total_mult_x + total_mult_y
    
    return total_mult_per_layer

# Constants
layers = 100
gates_per_layer = 1024

# Calculate total multiplications for one layer
mult_per_layer = calculate_mult_per_layer(layers, gates_per_layer)

# Calculate total multiplications for all layers
total_multiplications = layers * mult_per_layer

print(f"Total multiplications per layer: {mult_per_layer}")
print(f"Total multiplications for all layers: {total_multiplications}")


Total multiplications per layer: 3326
Total multiplications for all layers: 332600


In a typical acceleration, we can have more than one multiplier to better utilize the parallelism of Sum-check protocol. By bringing the number of multiplier into calculation, we can have

In [22]:
def calculate_time_per_layer(gates_per_layer, num_mult):
    log_gates = int(np.log2(gates_per_layer))
    
    # Step 1: Compute 'hg' table with parallel multipliers
    first_half = log_gates // 2
    second_half = log_gates - first_half
    mult_first_half = 2 ** first_half * (first_half - 1)
    mult_second_half = 2 ** second_half * (second_half - 1)
    mult_combine_halves = 2 ** first_half * 2 ** second_half
    
    total_mult_hg = mult_first_half + mult_second_half + mult_combine_halves
    time_step1 = total_mult_hg / num_mult
    
    # Step 2: Sumcheck for 'x' variables with limited multipliers
    sumcheck_times = []
    for i in range(log_gates - 1, -1, -1):
        needed_mult = 2 ** i
        if needed_mult > num_mult:
            rounds = np.ceil(needed_mult / num_mult)
            sumcheck_times.append(rounds)
        else:
            sumcheck_times.append(1)  # Only one round needed if multipliers suffice

    time_step2 = sum(sumcheck_times)
    
    # Step 3: Same as Step 2 for 'y' variables
    time_step3 = time_step2
    
    # Total time for one layer
    total_time_per_layer = time_step1 + time_step2 + time_step3
    
    return total_time_per_layer

# Constants
gates_per_layer = 1024
num_mult = 10  # Example number of multipliers available

# Calculate time for one layer
time_per_layer = calculate_time_per_layer(gates_per_layer, num_mult)

print(f"Time required per layer: {time_per_layer} rounds")

Time required per layer: 344.0 rounds


To calculate the total time required to complete all layers of multiplications on a 200 MHz system where each multiplication operation takes 100 cycles, we can adjust the previous function to account for this specific scenario. We will also multiply the results by the cycle time to convert the number of rounds into actual time.

In [23]:
import numpy as np

def calculate_total_cycles_all_layers(layers, gates_per_layer, num_mult, cycle_per_mult, frequency_mhz):
    log_gates = int(np.log2(gates_per_layer))
    cycle_period = 1 / (frequency_mhz * 1e6)  # Cycle time in seconds
    
    def calculate_cycles_per_layer():
        # Step 1: Compute 'hg' table with parallel multipliers
        first_half = log_gates // 2
        second_half = log_gates - first_half
        mult_first_half = 2 ** first_half * (first_half - 1)
        mult_second_half = 2 ** second_half * (second_half - 1)
        mult_combine_halves = 2 ** first_half * 2 ** second_half
        
        total_mult_hg = mult_first_half + mult_second_half + mult_combine_halves
        cycles_step1 = np.ceil(total_mult_hg / num_mult) * cycle_per_mult
        
        # Steps 2 and 3: Sumcheck for 'x' and 'y' variables with limited multipliers
        cycles_step2 = 0
        cycles_step3 = 0
        for i in range(log_gates - 1, -1, -1):
            needed_mult = 2 ** i
            cycle_count = np.ceil(needed_mult / num_mult) * cycle_per_mult
            cycles_step2 += cycle_count
            cycles_step3 += cycle_count  # Assuming Step 3 is similar to Step 2

        # Total cycles for one layer
        total_cycles = cycles_step1 + cycles_step2 + cycles_step3
        return total_cycles

    for layer in range(1, layers + 1):
        layer_cycles = calculate_cycles_per_layer()
        layer_time_micro_sec = layer_cycles * cycle_period * 1000
        if layer == 1:
            print(f"Cycles required for layer {layer}: {layer_cycles}")
            print(f"Micro second required for layer {layer}: {layer_time_micro_sec:.6f}")

# Constants
layers = 2
gates_per_layer = 1024
num_mult = 10        # A reasonable Number of wide multiplier to be implemented on chip
cycle_per_mult = 100 # Typical number of cycle latency for wide multiplier
frequency_mhz = 200  # System frequency in MHz

# Calculate and print the cycle counts for each layer
calculate_total_cycles_all_layers(layers, gates_per_layer, num_mult, cycle_per_mult, frequency_mhz)


Cycles required for layer 1: 34400.0
Micro second required for layer 1: 0.172000


From the calculations discussed above, we can observe that on a typical enterprise-grade FPGA, completing one layer of the GKR proof requires about 30,000 to 100,000 multiplication clock cycles when equipped with a moderate number of multipliers (10-15). In the case of sumcheck for 10 variables, at least 22 hash operations are necessary to generate all the random challenges. If we do not utilize a hardened hash function, and instead perform hash computations using a CPU soft core, the time required for hashing will be on the same order of magnitude as the time needed for multiplications (usually each hash function will take around 10,000 cycles on general purpose devices.). Therefore, it is essential to implement hardened SHA3-256 to efficiently generate random challenges.

### Quantitative Analysis

The calculations mentioned previously primarily aim to establish the order of magnitude for the performance of various components to achieve the desired performance metrics within the given hardware parameters. They are not actual performance indicators of a product. It's important to recognize the practical scenario where each layer of the GKR might require between 1 to 10 MB of data, assuming an average of 5 MB per layer. Considering the computation time for each layer is 100,000 cycles, and with a system frequency of 200 MHz, we need to calculate the necessary bandwidth.

To calculate the required bandwidth, we first determine the time per cycle and then the total time needed for 100,000 cycles at 200 MHz:

- **Cycle Time:** 1 \/ 200,000,000 seconds per cycle = $5e^{-9}$ seconds per cycle.
- **Total Time for 100,000 cycles:**  0.0005 seconds.

Now, we can calculate the bandwidth needed to load 5 MB of data in this time:

- **Data Size:** 5 MB = 5 × 10^6 bytes = 5 × 8 × 10^6 bits = 40 × 10^6 bits.
- **Bandwidth:** 40 \times 10^6  bits \/ 0.0005 seconds = 80,000,000,000 bits per second = 10 GBps.

Thus, to handle each layer's computation within the given time constraints on a 200 MHz system, a bandwidth of approximately 10 GBps is required.


## Conclusion and Takeaways

- There is no need to fully implement the entire GKR proof acceleration system in hardware. Due to the characteristics of the GKR algorithm, it is only necessary to design accelerators specifically for the SumCheck process.

- For each round of SumCheck, it's required to construct an on-chip SRAM with a capacity between 1-10 MB to store the current proof and the lookup tables needed for Linear GKR.

- It is essential to harden the random challenge generation functions. Relying on generic soft cores for this purpose would lead to computational bottlenecks, severely impacting the overall performance.

- Large integer multiplication constitutes the primary computational overhead. However, the number of multipliers does not solely determine the acceleration ratio; about ten multipliers (approximately the logarithm of the number of gates per layer) are sufficient due to the properties of SumCheck.

- The bandwidth requirement for each SumCheck proof acceleration core is likely to exceed 10 GB/s. Considering the limitations of bandwidth, strategically utilizing available bandwidth is crucial to achieving a high end-to-end acceleration ratio.

The intersection of cryptographic protocols like GKR with advanced hardware technologies such as FPGA and ASIC represents a pivotal development in the field of data security. By harnessing the power of specialized hardware, the implementation of zero-knowledge proofs can be made more efficient, paving the way for broader adoption and more robust applications in the digital world.

[Tha13](https://eprint.iacr.org/2013/351.pdf): Justin Thaler. Time-optimal interactive proofs for circuit evaluation. In Proceedings of the
International Cryptology Conference (CRYPTO), 2013.

[XZZ'19](https://eprint.iacr.org/2019/317.pdf): Xie, T., Zhang, J., Zhang, Y., Papamanthou, C., & Song, D. (2019). Libra: Succinct zero-knowledge proofs with optimal prover computation. In Advances in Cryptology–CRYPTO 2019: 39th Annual International Cryptology Conference, Santa Barbara, CA, USA, August 18–22, 2019, Proceedings, Part III 39 (pp. 733-764). Springer International Publishing.
